<a href="https://colab.research.google.com/github/JSJeong-me/AI-Innovation-2024/blob/main/NLP/4-5-RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain_community openai chromadb tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 728.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.5/383.5 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 604.0/604.0 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.9 MB/s eta 0:00

In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader

In [3]:
from google.colab import userdata
import os

userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
api_key = os.getenv("OPENAI_API_KEY")

In [4]:
!mkdir data
!wget https://github.com/JSJeong-me/GPT-Web/raw/main/images/paul_graham_essay.txt -O ./data/paul_graham_essay.txt

--2024-10-07 08:01:00--  https://github.com/JSJeong-me/GPT-Web/raw/main/images/paul_graham_essay.txt
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/JSJeong-me/GPT-Web/main/images/paul_graham_essay.txt [following]
--2024-10-07 08:01:01--  https://raw.githubusercontent.com/JSJeong-me/GPT-Web/main/images/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘./data/paul_graham_essay.txt’

./data/paul_graham_ 100%[===================>]  73.28K  --.-KB/s    in 0.02s   

2024-10-07 08:01:01 (4.71 MB/s) - ‘./data/paul_graham_essay.txt’ saved [75042/7

In [5]:
# Step 1: 문서 로드
# 여기에 분석하고자 하는 텍스트 파일을 넣으세요.
loader = TextLoader('./data/paul_graham_essay.txt')
documents = loader.load()


In [16]:
# Step 2: 텍스트 분할
from langchain.text_splitter import CharacterTextSplitter # import the CharacterTextSplitter class

# 문서를 최대 1000 토큰의 청크로 분할합니다.
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0) # reduced chunk size to 1000
documents = text_splitter.split_documents(documents)

# Step 2: 임베딩 및 Chroma 벡터 스토어 생성
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents, embeddings)

In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

# Step 3: OpenAI LLM 설정 (GPT-4 모델 사용)
# llm = OpenAI(model="gpt-4") # This line is using the incorrect API for this model
llm = ChatOpenAI(model="gpt-4") # Use ChatOpenAI for chat models

In [14]:


# Step 4: 검색 및 QA 체인 생성
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)


In [18]:

# Step 5: 질문에 답변
query = "How many legs does a cat have?"
answer = qa.run(query)
print(f"Answer: {answer}")


Answer: A cat has four legs.
